In [1]:
import cv2
from google.colab.patches import cv2_imshow

In [2]:
# Load video
vid_location = "vid.mp4"
cap = cv2.VideoCapture(vid_location)

# Load our classifiers
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml") 
l_eye_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_lefteye_2splits.xml")
r_eye_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_righteye_2splits.xml")
smile_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_smile.xml")

# Grab the width and height of the input video
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Create the video writer for MP4 format
out = cv2.VideoWriter('outpy.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 20.0, (frame_width,frame_height))

In [3]:
# Iterate while the video lasts
while cap.isOpened():

  # Read frame's
  ret, frame = cap.read()

  # Check if the video is still lasting
  if ret==True:

    # Convert frame to Grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Find faces in that frame
    # Parameters 
    #(1.1 -> scaleFactor : Parameter specifying how much the image size is reduced at each image scale)
    #(6 -> minNeighbors : Parameter specifying how many neighbors each candidate rectangle should have to retain it. This parameter will affect the quality of the detected faces)
    faces = face_classifier.detectMultiScale(gray, 1.1, 6)
    for (x, y, w, h) in faces:
      cv2.putText(frame, "Faces found: " + str(len(faces)), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (80, 0, 255), 3)

      # Draw a rectangle around the face
      cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

      # Select the region of the face, both gray and RGB
      roi_gray = gray[y:y+h, x:x+w]
      roi_color = frame[y:y+h, x:x+w]

      # Detect the left eye and draw a circle around it 
      l_eye = l_eye_classifier.detectMultiScale(roi_gray, 1.1, 10)
      for (ex, ey, ew, eh) in l_eye:
        cv2.circle(roi_color, ((ex + ex+ew)//2,  (ey + ey+eh)//2), 10, (255, 255, 0), 1)
      
      # Detect the right eye and draw a cricle around it also
      r_eye = r_eye_classifier.detectMultiScale(roi_gray, 1.1, 10)
      for (ex, ey, ew, eh) in r_eye:
        cv2.circle(roi_color, ((ex + ex+ew)//2,  (ey + ey+eh)//2), 10, (255, 0, 255), 1)

      # Detect smile and draw rectangle around it
      smile = smile_classifier.detectMultiScale(roi_gray, 1.8, 20)
      for (sx, sy, sw, sh) in smile:
        cv2.rectangle(roi_color, (sx, sy), (sx+sw, sy+sh), (255, 0, 0), 2)
      
      # Save the new video
    out.write(frame)
    print("Out!")
  else:
    break

# Close all opened files and save them
cap.release()
out.release()
cv2.destroyAllWindows()